In [1]:
%cd ..

/Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function/dataset_curation


In [2]:
import pandas as pd
import scripts

/Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function/venv/lib/python3.12/site-packages/goatools/__init__.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


/Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function/dataset_curation/scripts/../dependencies/go-basic.obo: fmt(1.2) rel(2025-07-22) 43,230 Terms


In [3]:
# ENV
from os import getenv
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
df = pd.read_csv(getenv("FETCHED_DATA"), index_col="Entry")

In [5]:
scripts.clean_all_cols(df, inplace=True)

No preprocessing rule for 'Sequence' — leaving as is.


,Domain [FT],Domain [CC],Protein families,Gene Ontology (molecular function),Gene Ontology (biological process),Function [CC],Catalytic activity,EC number,Pathway,Rhea ID,Cofactor,Sequence
Entry,,,,,,,,,,,,
P77499,2..246,NaN,"ABC transporter superfamily, Ycf16 family","(GO:0005524, GO:0016887)","(GO:0016226, GO:0009314)",has low atpase activity the sufbcd complex act...,NaN,NaN,NaN,NaN,NaN,mlsikdlhvsvedkailrglsldvhpgevhaimgpngsgkstlsat...
P0DP70,1..114,NaN,Binding-protein-dependent transport system per...,NaN,"(GO:0015716, GO:0055085)",c-terminal fragment of the phne protein part o...,NaN,NaN,NaN,NaN,NaN,mlalfihttgvlskllseaveaiepgpvegiratgankleeilygv...
Q8A8S8,35..222,NaN,LipB family,GO:0033819,GO:0036211,catalyzes the transfer of endogenously produce...,octanoyl-[ACP] + L-lysyl-[protein] = N(6)-octa...,2.3.1.181,protein modification protein lipoylation via e...,17665,NaN,mktvlvdwnlipyaeawqrqtewfdtlvrakaqgeayenriimceh...
P76278,NaN,NaN,CopD family,NaN,GO:0006825,NaN,NaN,NaN,NaN,NaN,NaN,mlaftwialrfihftslmlvfgfamygawlapltirrllakrflrl...
P77378,NaN,NaN,ETF alpha-subunit/FixB family,"(GO:0009055, GO:0050660)",GO:0033539,may play a role in a redox process,NaN,NaN,NaN,NaN,NaN,msqlnsvwvfsdnperyaelfggaqqwgqqvyaivqntdqaqavmp...
...,...,...,...,...,...,...,...,...,...,...,...,...
A0A3C0NZX8,3..125,NaN,NaN,GO:0016758,GO:0009058,NaN,NaN,NaN,NaN,NaN,NaN,mvsiivpvyqakeyiektirsveaqtfrdwelilvddcggdgslel...
A0A3R6DDG7,40..85,NaN,NaN,GO:0003677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,meivsiekktfemmvasfnalaekvaalrrrsdggrmerwltgeev...
A0A414IHG7,16..82,NaN,NaN,"(GO:0005524, GO:0030983)",GO:0006298,NaN,NaN,NaN,NaN,NaN,NaN,mtereliqqqqelgdsrflvtrsgkffsaygcgafalaratgyrvm...


In [10]:
df = df[df.notna().all(axis=1)]

# Feature Engineering

In [12]:
# Inits
txt_embedder = scripts.FreeTXTEmbedder(getenv("OPENAI_API_KEY"), "SMALL_OPENAI_MODEL",
                                       cache_file_path="testing_cache", caching_mode="CREATE/OVERRIDE")
aa_embedder = scripts.AAChainEmbedder("ESM2")

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Domain [FT]

In [14]:
df = scripts.embed_ft_domains(df, aa_embedder, inplace=False)

## Domain [CC], Function [CC], Catalytic activity, Pathway

In [16]:
df = scripts.embed_freetxt_cols(df, ["Domain [CC]", "Function [CC]", "Catalytic activity", "Pathway"], txt_embedder, inplace=False)

In [17]:
df.drop(columns=["Protein families"], inplace=True)

## Gene Ontology (molecular function), Gene Ontology (biological process)

In [19]:
df, _ = scripts.encode_go(df, "Gene Ontology (molecular function)", 0.8, inplace=False)
df, _ = scripts.encode_go(df, "Gene Ontology (biological process)", 0.8, inplace=False)

## EC number

In [23]:
df, _ = scripts.encode_ec(df, level=3, inplace=False)

## Rhea ID, Cofactor

In [21]:
df, _ = scripts.encode_multihot(df, "Rhea ID", inplace=False)
df, _ = scripts.encode_multihot(df, "Cofactor", inplace=False)

In [24]:
df

,Domain [CC],Gene Ontology (molecular function),Gene Ontology (biological process),Function [CC],Catalytic activity,EC number,Pathway,Rhea ID,Cofactor,domain_embedding
Entry,,,,,,,,,,
A0A174S1W4,"[0.015556952, 0.012779324, 0.03192031, 0.09134...",[1],[1],"[0.039770078, 0.023883486, 0.02093557, 0.05930...","[0.0010682484, -0.013508173, 0.018412925, 0.00...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[-0.039693218, -0.018673904, -0.005366514, 0.0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 1, 0]","[0.06519439, -0.59267235, 0.39969912, 0.265044..."
A1A0T4,"[0.034996994, 0.036172785, 0.0057117953, 0.026...",[1],[1],"[0.011446279, 0.029901857, 0.02459995, 0.02289...","[0.019794362, -0.0031235574, 0.007702409, 0.02...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]","[-0.001521531, 0.0015805489, 0.0067264284, 0.0...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 0, 0]","[0.07417185, -0.603595, 0.40701595, 0.2565485,..."
A0A0M6WK58,"[0.0073871133, 0.012897757, 0.06351492, 0.0227...",[1],[1],"[-0.010900132, 0.014565346, 0.03548214, 0.0166...","[0.019671533, -0.020075118, 0.004650996, 0.024...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.0042044334, 0.0016716522, 0.0033233939, 0.0...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0]","[0.07809408, -0.60110337, 0.40830672, 0.252053..."
R9IM47,"[0.034996994, 0.036172785, 0.0057117953, 0.026...",[1],[1],"[0.011446279, 0.029901857, 0.02459995, 0.02289...","[0.019794362, -0.0031235574, 0.007702409, 0.02...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]","[-0.001521531, 0.0015805489, 0.0067264284, 0.0...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 0, 0]","[0.07626842, -0.60418457, 0.40725395, 0.255790..."
F3AQF4,"[-0.011002625, -0.030456543, 0.042848732, 0.00...",[1],[1],"[0.022530414, 0.04027945, -0.0062423493, 0.068...","[0.024989761, 0.023087624, -0.018315194, 0.065...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0.005868458, 0.01230873, -0.026055835, 0.0351...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]","[1, 0, 0, 0, 0, 0]","[0.08569833, -0.600624, 0.42240095, 0.24795365..."
R8W4J3,"[0.0073871133, 0.012897757, 0.06351492, 0.0227...",[1],[1],"[-0.010900132, 0.014565346, 0.03548214, 0.0166...","[0.019671533, -0.020075118, 0.004650996, 0.024...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.0042044334, 0.0016716522, 0.0033233939, 0.0...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0]","[0.07836224, -0.6018773, 0.40831724, 0.2527847..."
A0A174LTZ8,"[0.0073871133, 0.012897757, 0.06351492, 0.0227...",[1],[1],"[-0.010900132, 0.014565346, 0.03548214, 0.0166...","[0.019671533, -0.020075118, 0.004650996, 0.024...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.0042044334, 0.0016716522, 0.0033233939, 0.0...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0]","[0.07836224, -0.6018773, 0.40831724, 0.2527847..."
Q54089,"[0.02599805, -0.0066759973, 0.0365628, 0.01812...",[1],[1],"[0.018219236, 0.029314283, 0.04195096, 0.01575...","[0.003807895, 0.017862596, 0.027607976, 0.0195...","[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]","[-0.029993352, 0.028223408, 0.032289494, 0.012...","[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 0, 0]","[0.09226795, -0.58187866, 0.43748796, 0.23752,..."
P11454,"[0.035087395, 0.0027331295, -0.017778553, 0.01...",[1],[1],"[0.03228834, -0.039987497, -0.0074284836, 0.03...","[0.006831822, -0.05309955, -0.028036198, 0.011...","[0, 0, 0, 0, 0, 0, 1, 1, 0, 0]","[0.010805607, -0.042125773, -0.014089606, 0.00...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[0, 0, 0, 0, 0, 1]","[0.10389228, -0.5899302, 0.4392348, 0.23114714..."


# NOTE:
# I know I had to use inplace=True, but Pandas was raising a bunch of warnings. I need to alter the logic mutating the df.
# This is because Pandas was thinking I was accidentally mutating the original df via view objects.